In [ ]:
# !pip install geopy

In [2]:
import cv2
import os 
import numpy as np
import pandas as pd
try:
    from PIL import Image
except ImportError:
    import Image
import pytesseract
import matplotlib.pyplot as plt
from skimage import util 
from skimage.color import rgb2gray

# Get geo

In [23]:
import geopy

locator = geopy.Nominatim(user_agent='myGeocoder')
location = locator.geocode('Champ de Mars, Paris, France')
location = locator.geocode('Альметьевск; Белоглазова 131')
print('Latitude = {}, Longitude = {}'.format(location.latitude, location.longitude))

Latitude = 54.90249145, Longitude = 52.2897052


In [109]:
# тут ручками начали форматировать и получили "Геолокация камер формат.xlsx"
# df = pd.read_excel('../Геолокация камер формат.xlsx',index_col='ID папки')
# print(df.Type.value_counts())
# df['lat']=np.nan
# df['lon']=np.nan
# for i,adress in enumerate(df['Адрес установки камеры'].values):
#     try: 
#         df['lat'][i] = locator.geocode(adress).latitude
#         df['lon'][i] = locator.geocode(adress).longitude
#     except:
#         pass
# df.to_csv('../geo_cam.csv',encoding='cp1251',sep=';')
# тут ручками начали забивать
# df = pd.read_csv('../geo_cam.csv',encoding='cp1251',sep=';',index_col='ID папки')
# for (idx, val) in df['lon'][df['lon'].isna()].iteritems():
#     lat,lon =  df.loc[idx,'lat'].split(', ')
#     df.loc[idx,'lon'] = lon
#     df.loc[idx,'lat'] = lat
# df.to_csv('geo_cam_done.csv')

,Адрес установки камеры,Type,lat,lon
ID папки,,,,
0,"Альметьевск, Белоглазова 131",ТКО,54.90249145,52.289705
1,"Альметьевск, Белоглазова 151",ТКО,54.90021055,52.287217
2,"Альметьевск, Джалиля 15",ТКО,54.8969668,52.296581
3,"Альметьевск, Гафиатуллина 29Б",ТКО,54.9055572,52.270927
4,"Альметьевск, Гафиатуллина 39",ТКО,54.90640845,52.266456
5,"Альметьевск, Гафиатуллина 45",ТКО,54.905825300000004,52.264366
6,"Альметьевск, Гафиатуллина 47 , подъезд 1",ТКО,54.906021,52.263795
7,"Альметьевск, Гафиатуллина 47, подъезд 2",ТКО,54.906161,52.263831
8,"Альметьевск, Гафиатуллина 51Б",ТКО,54.9067449,52.262942


# Извлечение районов

In [124]:
.reverse("52.509669, 13.376294")

In [12]:
import geopy
locator = geopy.Nominatim(user_agent='myGeocoder')
location = locator.reverse("55.784364, 49.185636")

In [20]:
location.address

'4А к2, улица Габдуллы Кариева, ЖК «Казань ХХI век», Советский район, Казань, городской округ Казань, Татарстан, Приволжский федеральный округ, 420081, Россия'

Location(4А к2, улица Габдуллы Кариева, ЖК «Казань ХХI век», Советский район, Казань, городской округ Казань, Татарстан, Приволжский федеральный округ, 420081, Россия, (55.78433085, 49.185579679308375, 0.0))

Location(11, улица Чапаева, Кадышево, Авиастроительный район, Казань, городской округ Казань, Татарстан, Приволжский федеральный округ, 420096, Россия, (55.8970477, 49.129442657771975, 0.0))

In [22]:
df = pd.read_csv('geo_cam_done.csv',index_col='ID папки')
# for i in df.iterrows():
#     print(i)
df['Район'] = np.nan
df['Управл'] = np.nan
for i in df.index[16:]:
    location = locator.reverse(f'{df.lat.values[i]}, {df.lon.values[i]}').address.split(', ')
    for element in location:
        if 'райо' in element.lower():
            df.loc[i,'Район'] = element
df.to_csv('../geo_cam_rai.csv',encoding='cp1251',sep=';')
df
    
    

,Адрес установки камеры,Type,lat,lon,Район,Управл
ID папки,,,,,,
0,"Альметьевск, Белоглазова 131",ТКО,54.902491,52.289705,NaN,NaN
1,"Альметьевск, Белоглазова 151",ТКО,54.900211,52.287217,NaN,NaN
2,"Альметьевск, Джалиля 15",ТКО,54.896967,52.296581,NaN,NaN
3,"Альметьевск, Гафиатуллина 29Б",ТКО,54.905557,52.270927,NaN,NaN
4,"Альметьевск, Гафиатуллина 39",ТКО,54.906408,52.266456,NaN,NaN
5,"Альметьевск, Гафиатуллина 45",ТКО,54.905825,52.264366,NaN,NaN
6,"Альметьевск, Гафиатуллина 47 , подъезд 1",ТКО,54.906021,52.263795,NaN,NaN
7,"Альметьевск, Гафиатуллина 47, подъезд 2",ТКО,54.906161,52.263831,NaN,NaN
8,"Альметьевск, Гафиатуллина 51Б",ТКО,54.906745,52.262942,NaN,NaN


In [30]:
for i in pd.read_csv('../macro_micro_analytics/geo_cam_rai.csv',encoding='cp1251',sep=';',header=None)[6].unique():
    print(i)

УК «ЖКХ Московский район»
УК «УЮТНЫЙ ДОМ ГРУПП»
УК «Управляющая компания Райжилуправления Советского района»
УК «ЖиК»
УК «ЖКХ Авиастроительного района»
УК «СТАРЫЙ ГОРОД»
УК «Управляющая компания Вахитовского района»
УК «УЮТСЕРВИС»
УК «УПРАВЛЯЮЩАЯ КОМПАНИЯ РАН СЕРВИС»


# OCR

In [ ]:
pytesseract.pytesseract.tesseract_cmd = r'.\OCR\tesseract'

In [20]:
def preproc(video):
    """
    на вход название видео, в результате созадет 
    папку с этим названием и по кадрово ложит туда 
    600 на 600 картинки. 
    В результате возращаем массивы: широты, долготы, скорости, Таймстемпы пандоса, и собсвтенно эти картинки 600:600
    """
    name = video.split('/')[-1].replace('.MOV','')
    os.mkdir(name)
    vidcap = cv2.VideoCapture(video)
    success,image = vidcap.read()
    pytesseract.pytesseract.tesseract_cmd = r'.\OCR\tesseract'
    count = 0
    n=[]
    r=[]
    speed = []
    time = []
    test = []

    while success:
        cv2.imwrite(f"{name}/frame{count}.jpg" , cropCenter(image,(600,600)))     # save frame as JPEG file      

        #       print('Read a new frame: ', success)
        if success:
            try:
                x = pytesseract.image_to_string(util.invert(image[1000:,:1000])).replace('\n','').split() 
                n.append(float(x[0][2:]))
                r.append(float(x[1][2:]))
                speed.append(float(x[2]))
                time.append(pd.Timestamp(x[4].replace('/','.')+' '+x[5]))
                test.append(cropCenter(image,(600,600)))
                
            except:
                print(count,'error',x)
        
        count += 1
        success,image = vidcap.read()
    return np.array(n),np.array(r),np.array(speed),np.array(time),np.array(test)

In [10]:
# vidcap = cv2.VideoCapture('7.MOV')
# success,image = vidcap.read()

In [12]:
# plt.imshow(cropCenter(image,(600,600)))

In [21]:
# plt.figure(figsize=(15,4))
# plt.imshow(rgb2gray(util.invert(image[1000:,:1000])))
# pytesseract.image_to_string(util.invert(image[1000:,:1000]))

In [ ]:
preproc('7.MOV')

97 error ['N:59.29025', 'E£:32.20875', '30', 'KM/H', '24/09/2020', 'Tiegieet']
98 error ['N:59.29025', 'E£:32.20875', '30', 'KM/H', '24/09/2020', 'Tiegieet']
99 error ['N:59.29025', 'E£:32.20875', '30', 'KM/H', '24/09/2020', 'Tiegieet']
100 error ['N:59.29025', 'E£:32.20875', '30', 'KM/H', '24/09/2020', 'Tiegieet']
101 error ['N:59.29025', 'E£:32.20875', '30', 'KM/H', '24/09/2020', 'Tiegieet']
102 error ['N:59.29025', 'E£:32.20875', '30', 'KM/H', '24/09/2020"', 'hiegieet']
103 error ['N:59.29025', 'E£:32.20875', '30', 'KM/H', '24/09/2020)', 'TieZiee', 'tT']
104 error ['N:59.29025', 'E£:32.20875', '30', 'KM/H', '24/09/2020)', 'TieZiee', 'tT']
105 error ['N:59.29025', 'E£:32.20375', '30', 'KM/H', '24/09/2020"', 'Ti', 's2iew']
106 error ['N:59.29025', 'E£:32.20375', '30', 'KM/H', '24/09/2020"', 'Ti', '2ieew']
107 error ['N:59.29025', 'E£:32.20375', '30', 'KM/H', '24/09/2020"', 'Tiare', 'ew']
108 error ['N:59.29025', 'E£:32.20375', '30', 'KM/H', '24/09/2020"', 'Tiare', 'ew']
109 error ['N:

In [25]:

# Simple image to string


In [29]:
x= tt
x 

['N:59.28997', 'R:32.20857', '26', 'KM/H', '24/09/2020', '11:21:27', 'CARCAM']

In [33]:
np.datetime64('24.09.2020 11:21:27')

ValueError: Error parsing datetime string "24.09.2020 11:21:27" at position 2

In [36]:
np.datetime64(pd.Timestamp('24.09.2020 11:21:27'))

numpy.datetime64('2020-09-24T11:21:27.000000')

In [41]:
r

32.20857

In [42]:
n

59.28997